In [1]:
#Import required libraries
import pandas as pd
import numpy as np

 # <span style="color:#346888;">TABLE OF CONTENTS</span>


* **[Load Data](#load-data)**


* **[Understanding Data](#un-data)**


* **[Data cleaning and preparation ](#clean-data)**
    * **[Drop columns](#drop-cols)**
    * **[Missing data](#missing-rows)**
    * **[Data aggregation](#data-agg)**
    
    
* **[Understanding the trends](#trend)**


* **[Analysing the combined trends](#trend-com)**
    * **[Average closing value](#close-val)**
    * **[Closing volume](#vol)**
    
    
* **[Correlation](#corr)**

## <span style="color:#346888;">Load Data</span> <a id="load-data"></a>

In [2]:
#load bitcoin data
btc_master = pd.read_csv('../input/gemini_BTCUSD_2020_1min.csv')

#load ethereum data
eth_master = pd.read_csv('../input/gemini_ETHUSD_2020_1min.csv')

#load litecoin data
ltc_master = pd.read_csv('../input/gemini_LTCUSD_2020_1min.csv')

## <span style="color:#346888;">Understanding Data</span> <a id="un-data"></a>

In [3]:
#Check the data dimensions for bitcoin
btc_master.head()

Unix Timestamp             Date  Symbol      Open      High       Low  \
0    1.618880e+12   4/20/2021 0:02  BTCUSD  55717.47  55723.00  55541.69   
1    1.618880e+12   4/20/2021 0:01  BTCUSD  55768.94  55849.82  55711.74   
2    1.618880e+12   4/20/2021 0:00  BTCUSD  55691.79  55793.15  55691.79   
3    1.618880e+12  4/19/2021 23:59  BTCUSD  55777.86  55777.86  55677.92   
4    1.618880e+12  4/19/2021 23:58  BTCUSD  55803.50  55823.88  55773.08   

      Close    Volume  
0  55541.69  2.406734  
1  55717.47  0.573458  
2  55768.94  3.309904  
3  55691.79  0.684934  
4  55777.86  0.477225

In [4]:
btc_master.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 658065 entries, 0 to 658064
Data columns (total 8 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   Unix Timestamp  658065 non-null  float64
 1   Date            658065 non-null  object 
 2   Symbol          658065 non-null  object 
 3   Open            658065 non-null  float64
 4   High            658065 non-null  float64
 5   Low             658065 non-null  float64
 6   Close           658065 non-null  float64
 7   Volume          658065 non-null  float64
dtypes: float64(6), object(2)
memory usage: 40.2+ MB


In [5]:
#Check the data dimensions for ethereum
eth_master.shape

(658059, 8)

In [6]:
eth_master.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 658059 entries, 0 to 658058
Data columns (total 8 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   Unix Timestamp  658059 non-null  float64
 1   Date            658059 non-null  object 
 2   Symbol          658059 non-null  object 
 3   Open            658059 non-null  float64
 4   High            658059 non-null  float64
 5   Low             658059 non-null  float64
 6   Close           658059 non-null  float64
 7   Volume          658059 non-null  float64
dtypes: float64(6), object(2)
memory usage: 40.2+ MB


In [7]:
#Check the data dimensions for litecoin
ltc_master.shape

(658011, 8)

In [8]:
ltc_master.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 658011 entries, 0 to 658010
Data columns (total 8 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   Unix Timestamp  658011 non-null  float64
 1   Date            658011 non-null  object 
 2   Symbol          658011 non-null  object 
 3   Open            658011 non-null  float64
 4   High            658011 non-null  float64
 5   Low             658011 non-null  float64
 6   Close           658011 non-null  float64
 7   Volume          658011 non-null  float64
dtypes: float64(6), object(2)
memory usage: 40.2+ MB


Litecoin data is 54 records short compared to bitcoin and 48 records short compared to ethereum data.

##### Looking at the dataframe info there are no null values in any dataset

## <span style="color:#346888;">Data cleaning and preparation</span> <a class="anchor" id="clean-data"></a>

In [9]:
#Convert the object type Date column to datetime type
btc_master['Date'] = pd.to_datetime(btc_master['Date'], format='%m/%d/%Y %H:%M')
eth_master['Date'] = pd.to_datetime(eth_master['Date'], format='%m/%d/%Y %H:%M')
ltc_master['Date'] = pd.to_datetime(ltc_master['Date'], format='%m/%d/%Y %H:%M')

### <span style="color:#346888;">Drop unnecessary columns</span> <a id="drop-cols"></a>

As we are trying to understand the correlation between the currencies over a year, the columns of intrest will be `Date`, `close` and `volume`. Apart from these three columns we can drop rest of the data from the dataset.

In [10]:
#drop from bitcoin
btc_master.drop(['Unix Timestamp','Symbol','Open', 'High','Low'], axis = 1, inplace = True)
#drop from ethereum 
eth_master.drop(['Unix Timestamp','Symbol','Open', 'High','Low'], axis = 1, inplace = True)
#drop from litecoin
ltc_master.drop(['Unix Timestamp','Symbol','Open', 'High','Low'], axis = 1, inplace = True)

ltc_master.columns

Index(['Date', 'Close', 'Volume'], dtype='object')

### <span style="color:#346888;">Find missing data</span> <a id="missing-rows"></a>

As we noticed that not all three datasets have the same number of records, let us analyse and understand what data is missing.

In [11]:
#get all the records in bitcoin dataset but not in ethereum dataset
btc_master[~(btc_master['Date'].isin(eth_master["Date"]))]

Date     Close     Volume
12952  2021-04-11 00:08:00  59757.12   0.007522
14393  2021-04-10 00:03:00  58115.69   0.043307
43191  2021-03-21 00:04:00  58120.76   7.889050
63349  2021-03-07 00:06:00  49109.05   0.032708
93581  2021-02-14 00:08:00  47252.35   0.119529
103657 2021-02-07 00:07:00  39158.21   2.360374
109416 2021-02-03 00:04:00  35457.78   0.818440
113734 2021-01-31 00:06:00  34328.75   2.480243
123813 2021-01-24 00:05:00  31781.39   0.083153
143970 2021-01-10 00:04:00  40499.99   0.164603
187164 2020-12-11 00:03:00  18265.92   0.000000
192925 2020-12-06 00:03:00  19254.31  30.687611
241886 2020-11-01 00:03:00  13779.51   0.001698
253405 2020-10-24 00:03:00  12942.71   0.146458
309565 2020-09-14 00:01:00  10330.47   0.105931
390210 2020-07-17 00:00:00   9133.49   0.000000
609099 2020-02-07 00:02:00   9773.11   0.000000
646541 2020-01-10 00:01:00   7820.01   0.000000

In [12]:
#get all the records in ethereum dataset but not in bitcoin dataset
eth_master[~(eth_master['Date'].isin(btc_master["Date"]))]

Date    Close    Volume
0      2021-04-20 00:03:00  2154.38  0.701288
12953  2021-04-11 00:04:00  2130.88  0.000000
93579  2021-02-14 00:03:00  1818.84  0.269269
103655 2021-02-07 00:03:00  1675.69  2.573782
174201 2020-12-20 00:02:00   656.70  0.000000
251959 2020-10-25 00:02:00   412.22  0.000000
289398 2020-09-29 00:01:00   352.50  0.000000
295159 2020-09-24 00:02:00   320.00  0.000000
341243 2020-08-20 00:01:00   408.61  0.000000
503971 2020-04-23 00:01:00   183.37  0.000000
591813 2020-02-19 00:03:00   282.23  0.000000
627814 2020-01-23 00:03:00   167.77  0.000000

In [13]:
#get all the records in litecoin dataset but not in bitcoin dataset
ltc_master[~(ltc_master['Date'].isin(btc_master["Date"]))].count()

Date      47
Close     47
Volume    47
dtype: int64

In [14]:
#get all the records in litecoin dataset but not in ethereum dataset
ltc_master[~(ltc_master['Date'].isin(eth_master["Date"]))].count()

Date      41
Close     41
Volume    41
dtype: int64

Looking at the records missing, there does not seem to be any obvious pattern as to why the data is missing. We have the data by the minute of the hour, for understanding the correlation we plan to aggregate the currency data by day. As we do not have more than a couple of minutes data missing for some days the missing data proportion is negligible in this usecase. Hence, the missing values need not be treated.

### <span style="color:#346888;">Data aggregation</span> <a id="data-agg"></a>

##### <span style="color:#346888;">Bitcoin</span>

In [15]:
#Loose the time information
btc_master['Date'] = btc_master['Date'].dt.date

#Aggregate by date
btc_grouped = pd.pivot_table(btc_master, values=['Close', 'Volume'], index=['Date'],
                    aggfunc={'Close': np.mean,
                             'Volume': np.sum})

btc_grouped.head()

Close       Volume
Date                                
2020-01-01  7195.641243   776.641615
2020-01-02  7072.362910   984.141603
2020-01-03  7215.680840  1292.330594
2020-01-04  7320.664035   402.636843
2020-01-05  7429.535437   554.773602

In [16]:
btc_grouped.shape

(476, 2)

##### <span style="color:#346888;">Ethereum</span>

In [17]:
#Loose the time information
eth_master['Date'] = eth_master['Date'].dt.date

#Aggregate by date
eth_grouped = pd.pivot_table(eth_master, values=['Close', 'Volume'], index=['Date'],
                    aggfunc={'Close': np.mean,
                             'Volume': np.sum})
eth_grouped.head()

Close        Volume
Date                                
2020-01-01  130.576681  10641.079145
2020-01-02  128.530792  11842.451144
2020-01-03  130.885833  22800.226361
2020-01-04  133.554104   7537.048701
2020-01-05  136.036493   7015.126626

In [18]:
eth_grouped.shape

(476, 2)

##### <span style="color:#346888;">Litecoin</span>

In [19]:
#Loose the time information
ltc_master['Date'] = ltc_master['Date'].dt.date

#Aggregate by date
ltc_grouped = pd.pivot_table(ltc_master, values=['Close', 'Volume'], index=['Date'],
                    aggfunc={'Close': np.mean,
                             'Volume': np.sum})
ltc_grouped.head()

Close       Volume
Date                              
2020-01-01  41.603819  1149.199683
2020-01-02  40.708660  5119.431567
2020-01-03  41.014042  6299.303403
2020-01-04  42.302889  1828.613739
2020-01-05  43.264583  6388.375047

In [20]:
ltc_grouped.shape

(476, 2)

<u>Once the data is aggregated we can see we have data for all the three currencies for 476 days.</u>

## <span style="color:#346888;">Understanding the trends</span> <a id="trend"></a>

Let's ask some questions to understand the trends

#### What is the highest average closing value between Jan-2020 and 2021-04-14?

In [21]:
btc = btc_grouped['Close'].max()
eth = eth_grouped['Close'].max()
ltc = ltc_grouped['Close'].max()

#What is the highest closing value of Bitcoin and when
print("Highest average value of BTC ", btc,"was recorded on ",btc_grouped[btc_grouped['Close'] == btc].index.values[0])

#What is the highest closing value of Ethereum and when
print("Highest average value of ETH ", eth,"was recorded on ",eth_grouped[eth_grouped['Close'] == eth].index.values[0])

#What is the highest closing value of Litecoin and when
print("Highest average value of LTC ", ltc,"was recorded on ",ltc_grouped[ltc_grouped['Close'] == ltc].index.values[0])


Highest average value of BTC  63451.637131944444 was recorded on  2021-04-14
Highest average value of ETH  2461.6089722222223 was recorded on  2021-04-15
Highest average value of LTC  318.223125 was recorded on  2021-04-17


The value of bitcoin is far higher compared to its counterparts. 

#### What is the highest volume transacted in one day between Jan-2020 and 2021-04-14?

In [22]:
btc = btc_grouped['Volume'].max()
eth = eth_grouped['Volume'].max()
ltc = ltc_grouped['Volume'].max()


#What is the highest closing volume of Bitcoin and when
print("Highest volume of BTC ", btc,"was recorded on ",
      btc_grouped[btc_grouped['Volume'] == btc].index.values[0])

#What is the highest closing volume of Ethereum and when
print("Highest volume of ETH ", eth,"was recorded on ",
     eth_grouped[eth_grouped['Volume'] == eth].index.values[0])

#What is the highest closing volume of Litecoin and when
print("Highest volume of LTC ", ltc,"was recorded on ",
     ltc_grouped[ltc_grouped['Volume'] == ltc].index.values[0])

Highest volume of BTC  18928.801040246 was recorded on  2020-03-13
Highest volume of ETH  206706.02655481 was recorded on  2020-03-12
Highest volume of LTC  204353.8343832 was recorded on  2021-01-11


Far more number of ethereum and litecoins have been transacted compared to bitcoins

#### What was the average closing value when the highest volume was transacted between Jan-2020 and 2021-04-14

In [23]:
btc = btc_grouped[btc_grouped['Volume'] == btc_grouped['Volume'].max()]['Close'].values[0]
eth = eth_grouped[eth_grouped['Volume'] == eth_grouped['Volume'].max()]['Close'].values[0]
ltc = ltc_grouped[ltc_grouped['Volume'] == ltc_grouped['Volume'].max()]['Close'].values[0]

#What is the closing value of Bitcoin and when the highest volume was transacted 
print("Highest volume of BTC was", btc_grouped['Volume'].max(),"and mean closing value on that day was",btc)

#What is the highest closing volume of Ethereum and when
print("Highest volume of ETH ", eth_grouped['Volume'].max(),"and mean closing value on that day was ",eth)

#What is the highest closing volume of Litecoin and when
print("Highest volume of LTC ", ltc_grouped['Volume'].max(),"and mean closing value on that day was ",ltc)


Highest volume of BTC was 18928.801040246 and mean closing value on that day was 5313.687326388889
Highest volume of ETH  206706.02655481 and mean closing value on that day was  154.3535347222222
Highest volume of LTC  204353.8343832 and mean closing value on that day was  139.52608333333333


#### Was the average closing value at its lowest when highest volume was transacted

In [24]:
#What is the closing value of Bitcoin and when the highest volume was transacted 
print("Lowest average closing value of BTC ", btc_grouped['Close'].min())

#What is the closing value of Bitcoin and when the highest volume was transacted 
print("Lowest average closing value of ETH ", eth_grouped['Close'].min())

#What is the closing value of Bitcoin and when the highest volume was transacted 
print("Lowest average closing value of LTC ", ltc_grouped['Close'].min())

Lowest average closing value of BTC  4987.150173611111
Lowest average closing value of ETH  112.70609027777778
Lowest average closing value of LTC  33.027368055555556


Interestingly, the highest volumn was not transacted when the average closing value was at its least.

#### How did the average closing value vary by time for bitcoin

## <span style="color:#346888;">Correlation</span> <a id="corr"></a>

Let us merge all the three data sets to analyse the correlation of average closing value and the volume of all the three currencies

In [25]:
#Merge bitcoin and ethereum data
btc_eth = pd.merge(btc_grouped, eth_grouped, suffixes=('_btc', '_eth'), left_index=True, right_index=True)
btc_eth.head()

Close_btc   Volume_btc   Close_eth    Volume_eth
Date                                                          
2020-01-01  7195.641243   776.641615  130.576681  10641.079145
2020-01-02  7072.362910   984.141603  128.530792  11842.451144
2020-01-03  7215.680840  1292.330594  130.885833  22800.226361
2020-01-04  7320.664035   402.636843  133.554104   7537.048701
2020-01-05  7429.535437   554.773602  136.036493   7015.126626

In [26]:
#Merge litecoin with other two
btc_eth_ltc = pd.merge(btc_eth, ltc_grouped, left_index=True, right_index=True)
btc_eth_ltc.rename(columns={"Close": "Close_ltc", "Volume": "Volume_ltc"}, inplace=True)
btc_eth_ltc.head()

Close_btc   Volume_btc   Close_eth    Volume_eth  Close_ltc  \
Date                                                                        
2020-01-01  7195.641243   776.641615  130.576681  10641.079145  41.603819   
2020-01-02  7072.362910   984.141603  128.530792  11842.451144  40.708660   
2020-01-03  7215.680840  1292.330594  130.885833  22800.226361  41.014042   
2020-01-04  7320.664035   402.636843  133.554104   7537.048701  42.302889   
2020-01-05  7429.535437   554.773602  136.036493   7015.126626  43.264583   

             Volume_ltc  
Date                     
2020-01-01  1149.199683  
2020-01-02  5119.431567  
2020-01-03  6299.303403  
2020-01-04  1828.613739  
2020-01-05  6388.375047

In [27]:
#Corelation between the currencies
btc_eth_ltc.corr()

Close_btc  Volume_btc  Close_eth  Volume_eth  Close_ltc  \
Close_btc    1.000000    0.019898   0.987148    0.230203   0.978353   
Volume_btc   0.019898    1.000000   0.023653    0.799471   0.018683   
Close_eth    0.987148    0.023653   1.000000    0.225287   0.975283   
Volume_eth   0.230203    0.799471   0.225287    1.000000   0.258728   
Close_ltc    0.978353    0.018683   0.975283    0.258728   1.000000   
Volume_ltc   0.450823    0.639422   0.455818    0.736889   0.495338   

            Volume_ltc  
Close_btc     0.450823  
Volume_btc    0.639422  
Close_eth     0.455818  
Volume_eth    0.736889  
Close_ltc     0.495338  
Volume_ltc    1.000000